<a href="https://colab.research.google.com/github/Nid989/LLMs-Overview/blob/main/langchain/functions_tools_and_agents_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install openai python-dotenv langchain tiktoken
!pip install pydantic==1.10.8
!pip install langchain[docarray]
!pip install wikipedia

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(filename="./.env.txt"))
openai.api_key = os.environ['OPENAI_API_KEY']

#### `OpenAI: Function Calls`

In [3]:
import json

# Example dummy function hard coded to return the same weather
# IN production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"]
    }
    return json.dumps(weather_info)

In [4]:
# define a function (`As proposed by OpenAI`)
# `description` parameter (both) are really important, since this will be passed directly
# to the LLM and the model is gonna use this description to determine whether to use the function :)
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Fransico, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [5]:
messages = [
    {
        "role": "user",
        "content": "what's the weather like in Boston?"
    }
]

In [6]:
import openai

In [7]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [8]:
print(response)

ChatCompletion(id='chatcmpl-8Nq0rkKe4Tj673swdBsZNWKLqSzNS', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1700692173, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=84, total_tokens=102))


In [9]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [10]:
response_message.content # None

In [11]:
response_message.function_call

FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')

In [12]:
args = json.loads(response_message.function_call.arguments)
args

{'location': 'Boston, MA'}

In [13]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [14]:
# w/ OpenAI Function Calls, it doesnot utilize the defined function to generate the response
# rather it will tell us what function to call and what the arguments to the specific function should be!

In [15]:
# what if the message passed to the OpenAI model API is not related to the already defined function at all.
messages = [
    {
        "role": "user",
        "content": "hi!"
    }
]

In [16]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [17]:
print(response) # output `message` includes not None content and no function call argument as seen in the previous response

ChatCompletion(id='chatcmpl-8Nq0uuNh4Qayr8deb2uWORIGLWh6h', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1700692176, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=78, total_tokens=88))


In [18]:
# `function_call` parameter either to force function call or not
# mode: `auto`
messages = [
    {
        "role": "user",
        "content": "hi!"
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto" # default (i.e. the LLM choose b/w functions and self-response)
)
print(response)

ChatCompletion(id='chatcmpl-8Nq0xVEjw10jAP6wCjOTw9N79q0t3', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1700692179, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=78, total_tokens=88))


In [19]:
# mode: `none` # force LLM not make a function call
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?"
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none"
)
print(response)

ChatCompletion(id='chatcmpl-8Nq11qGZeTBWTrxy8BHsAnXv16gcK', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The weather in Boston is currently unavailable.', role='assistant', function_call=None, tool_calls=None))], created=1700692183, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=84, total_tokens=92))


In [20]:
# mode: assign a specific function (i.e. force the model to compulsorily use the function-call w/ function name provided)
messages = [
    {
        "role": "user",
        "content": "hi!"
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
print(response)

ChatCompletion(id='chatcmpl-8Nq1HolNNR6bN3CvQq9Yl0ebzJnU7', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "San Francisco, CA"\n}', name='get_current_weather'), tool_calls=None))], created=1700692199, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=85, total_tokens=97))


In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
print(response)

ChatCompletion(id='chatcmpl-8Nq1LlSP25CEb7GBZeWupYA4Iqx2M', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1700692203, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=90, total_tokens=100))


In [22]:
# NOTE: under the `usage` output parameter, we can see that w/ and w/o `functions` as input-argument
# the prompt_tokens utilization differs drastically, from 85 (w/ function-calling) to 15 (w/o function calling)
# since extra tokens are utilized to describe the function-calling procedure to the OpenAI LLM API.

# This can put constraint, since we have a limit over the OpenAI `prompt_tokens`, thus now we do have to keep
# track of the total input tokens already used by function-calling proc. alongwith the messages supplied

In [23]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?"
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto"
)
print(response)

ChatCompletion(id='chatcmpl-8Nq1NvrHtL0n9eEF48WDLjCxvXbKd', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1700692205, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=83, total_tokens=101))


In [24]:
messages.append(response.choices[0].message) # appending to the list of messages to simulate calling to function w/ args

In [25]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

In [26]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": observation
    }
)
print(messages)

[{'role': 'user', 'content': "What's the weather in Boston?"}, ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None), {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]


In [27]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages
)
print(response)

ChatCompletion(id='chatcmpl-8Nq1mIZOwKcs02GpmmSVM80wMOIOn', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The weather in Boston is currently sunny and windy with a temperature of 72°F.', role='assistant', function_call=None, tool_calls=None))], created=1700692230, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=76, total_tokens=93))


#### `LangChain Expression Language (LECL)`

In [28]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser  # will take a ChatMessage and convert to string

##### `Simple Chain`

In [29]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [30]:
chain = prompt | model | output_parser

In [31]:
chain.invoke({"topic": "bears"})

"Why don't bears like fast food?\n\nBecause they can't catch it!"

##### `More Complex Chain and Runnnable Map (to supply user-provided inputs to prompt)`

In [32]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [33]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [34]:
retriever.get_relevant_documents("Where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [35]:
retriever.get_relevant_documents("What do bears like to eat?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [36]:
template = """Answer the question based only on the following context:
{context}

Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

In [37]:
# How are we going to build the chain?
# i. We want the first and only input to the chain to be the user question.
# ii. Then we want to fetch the relevant document for the context which will be passed to the prompt
# iii. Finally, we want to pass the formulated prompt to the model and the output-parser later on.

In [38]:
# i. we want to create `Something` which takes a single question and turns it into
# a dictionary of 2 elements, `context` and `question`.
from langchain.schema.runnable import RunnableMap

In [39]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [40]:
chain.invoke({"question": "Where did harrison work?"})

'Harrison worked at Kensho.'

In [41]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [42]:
inputs.invoke({"question": "Where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'Where did harrison work?'}

##### `Bind and OpenAI Functions`

In [43]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given at airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    }
]

In [44]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [45]:
runnable = prompt | model

In [46]:
runnable.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'weather_search'}})

In [47]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given at airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sport events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The sports team to search for"
                },
            },
            "required": ["team_name"]
        }
    }
]

In [48]:
model = model.bind(functions=functions)

In [49]:
runnable = prompt | model

In [50]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "team_name": "patriots"\n}', 'name': 'sports_search'}})

##### `Fallbacks`
`with LangChain, we can attach fallbacks not only to the model but any component of the chain or the chain as whole.`

In [51]:
# let's say we want to output a JSON formatted string from the LLM
# NOTE: using an older version of OpenAI LLM API just to showcase the need for `fallbacks`.
from langchain.llms import OpenAI
import json

In [52]:
simple_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model="text-davinci-001"
)
simple_chain = simple_model | json.loads

In [53]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [54]:
simple_model.invoke(challenge)

'\n\n["The Waste Land","T.S. Eliot","April is the cruelest month, breeding lilacs out of the dead land"]\n\n["The Raven","Edgar Allan Poe","Once upon a midnight dreary, while I pondered, weak and weary"]\n\n["Ode to a Nightingale","John Keats","Thou still unravish\'d bride of quietness, Thou foster-child of silence and slow time"]'

In [55]:
# calling the below line, we get the JSONDecodeError, since the output string
# eventhough it has some structure, cannot be decoded by json.loads() function.
# simple_chain.invoke(challenge)

In [56]:
model = ChatOpenAI(temperature=0) # using a newer version of OpenAI models
chain = model | StrOutputParser() | json.loads

In [57]:
chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [58]:
# so we can start with a simple chain and attach fallbacks with `.with_fallbacks`
# so first the core runnable `simple_chain` will try to run and if error is raised
# then it goes through list of runnables provided as fallbacks, in this case its just
# 1 extra runnable `chain`.
final_chain = simple_chain.with_fallbacks([chain])

In [59]:
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the walls, a secret ballet'}}

In [60]:
import langchain
langchain.debug = True
final_chain.invoke(challenge)

[chain/start] [1:chain:RunnableWithFallbacks] Entering Chain run with input:
{
  "input": "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
}
[chain/start] [1:chain:RunnableWithFallbacks > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
}
[llm/start] [1:chain:RunnableWithFallbacks > 2:chain:RunnableSequence > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
  ]
}
[llm/end] [1:chain:RunnableWithFallbacks > 2:chain:RunnableSequence > 3:llm:OpenAI] [1.67s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\n[\"The Waste Land\",\"T.S. Eliot\",\"April is the cruelest month, breeding lilacs out of the dead land\"]\n\n[\"The Raven\",\"Edgar Allan Poe\",\"Once upon a midnig

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the walls, a secret ballet'}}

In [61]:
langchain.debug = False

##### `Interface`

In [62]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [63]:
# We will explore few different elements of the interface

In [65]:
chain.invoke({"topic": "bears"}) # synchronous method that calls it on 1 single input.

"Why don't bears wear shoes?\n\nBecause they already have bear feet!"

In [66]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}]) # synchronous method that calls it on list of inputs.

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 "Why don't frogs like to play basketball?\n\nBecause they always get too jumpy!"]

In [67]:
# stream back the output based on tokens generated (ideal for showcasing a streaming property when recording bot-replies in chat-interface)
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 socks
?
 


Because
 they
 have
 bear
 feet
!



In [68]:
# calls it in a asynchronous manner
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

#### `OpenAI Function Calling in LangChain`

In [69]:
# Objective: How to use OpenAI functions and combine them with LCEL (LangChain Expression Language)
# Additionally, explore pydantic which makes it easier to work with OpenAI functions.

In [70]:
from typing import List
from pydantic import BaseModel, Field

##### `Pydantic Syntax`
`Pydantic data classes are a blend of Python's data classes with the validation power of pydantic (since in normal python we specify data-types for each attribute but they're not validated and are just used for user reference)`

`They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.`

In [71]:
# in standard python you would create a class like this:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [72]:
foo = User(name="Joe", age=32, email="joe@gmail.com")
foo.name

'Joe'

In [73]:
foo = User(name="Joe", age="bar", email="joe@gmail.com")
foo.age # expected data-type for attr. `age` was "int" but the class cannot validate and accepted "str" too.

'bar'

In [74]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [75]:
foo_p = pUser(name="Joe", age=32, email="joe@gmail.com")
foo_p.name

'Joe'

In [76]:
# Note: below code will give ValidationError, since attr. `age` is supposed to be "int"
# foo_p = pUser(name="Joe", age="bar", email="joe@gmail.com")

In [77]:
# with Pydantic we can nest these data-structures
class Class(BaseModel):
    students: List[pUser]

In [78]:
obj = Class(
    students=[pUser(name="Joe", age=32, email="joe@gmail.com")]
)

In [79]:
obj

Class(students=[pUser(name='Joe', age=32, email='joe@gmail.com')])

##### `Pydantic to OpenAI function definition`

In [80]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [81]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [82]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [83]:
weather_function # the main description is pulled from the DocString of the class definition

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [84]:
class WeatherSearch(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [85]:
# DocString is a required **compulsory when defining a class definition inherited from Pydantic BaseModel
# Note: the below code will produce error a `KeyError` on calling `convert_pydantic_to_openai_function`
# since the DocString is not provided
# convert_pydantic_to_openai_function(WeatherSearch)

In [86]:
# However, when defining the attributes it is not compulsory to provide the description
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [87]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch2',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code']}}

In [88]:
# combining OpenAI function with LCEL
from langchain.chat_models import ChatOpenAI

In [89]:
model = ChatOpenAI()

In [90]:
model.invoke("what is the weather in SF today?", functions=[weather_function]) # w/ keyword args `functions`

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [91]:
# we can bind the function invocation to the model,
# reason: we can pass that {model, function} pair together and
# do not have to worry about the explicit mentioning of kwargs
model_with_function = model.bind(functions=[weather_function])

In [92]:
model_with_function.invoke("what is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

##### `Forcing it to use a function`

In [93]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name": "WeatherSearch"})

In [94]:
model_with_forced_function.invoke("what is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [95]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "LAX"\n}', 'name': 'WeatherSearch'}})

##### `Using in a Chain`

In [96]:
from langchain.prompts import ChatPromptTemplate

In [97]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [98]:
chain = prompt | model_with_function

In [99]:
chain.invoke({"input": "what is the weather in SF?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

##### `Using multiple functions`

In [100]:
# we can pass a set of functions and let the LLM decide which to use based on the question context.

In [101]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of the artisit to look up")
    n: int = Field(description="number of results")

In [102]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch)
]

In [103]:
model_with_functions = model.bind(functions=functions)

In [104]:
model_with_functions.invoke("what is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [105]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "artist_name": "taylor swift",\n  "n": 3\n}', 'name': 'ArtistSearch'}})

In [106]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')

#### `Tagging and Extraction`
`Allows to extract structured data from unstructured text`

In [107]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

##### `Tagging`

In [108]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg` or `neutral`") # through the tags (i.e. `pos`, `neg`, `neutral`) we're telling the LLM how the data should be structured,
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [109]:
convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'title': 'Tagging',
  'description': 'Tag the piece of text with particular info.',
  'type': 'object',
  'properties': {'sentiment': {'title': 'Sentiment',
    'description': 'sentiment of text, should be `pos`, `neg` or `neutral`',
    'type': 'string'},
   'language': {'title': 'Language',
    'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language']}}

In [110]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [111]:
model = ChatOpenAI(temperature=0.)

In [112]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [113]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [114]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [115]:
tagging_chain = prompt | model_with_functions

In [116]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "pos",\n  "language": "en"\n}', 'name': 'Tagging'}})

In [117]:
tagging_chain.invoke({"input": "no mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "neg",\n  "language": "it"\n}', 'name': 'Tagging'}})

In [118]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [119]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [120]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

##### `Extraction`
`Extraction is similar to tagging, but used for extracting multiple piece of information`

In [121]:
from typing import Optional
class Person(BaseModel):
    """Information about a Person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [122]:
class Information(BaseModel):
    """Information to extract"""
    people: List[Person] = Field(description="List of info about people")

In [123]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract',
 'parameters': {'title': 'Information',
  'description': 'Information to extract',
  'type': 'object',
  'properties': {'people': {'title': 'People',
    'description': 'List of info about people',
    'type': 'array',
    'items': {'title': 'Person',
     'description': 'Information about a Person.',
     'type': 'object',
     'properties': {'name': {'title': 'Name',
       'description': "person's name",
       'type': 'string'},
      'age': {'title': 'Age',
       'description': "person's age",
       'type': 'integer'}},
     'required': ['name']}}},
  'required': ['people']}}

In [124]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [125]:
extraction_model.invoke("Joe is 30, his mom is Martha") # model seems to think when it doenot know a person's age, then "age=0", but we can do better!

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha",\n      "age": 0\n    }\n  ]\n}', 'name': 'Information'}})

In [126]:
# Note: below prompt makes sure via `system-prior` that when a particular
# attribute value is unknown, then ignore that attribute for that instance.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("user", "{input}")
])

In [127]:
extraction_chain = prompt | extraction_model

In [128]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha"\n    }\n  ]\n}', 'name': 'Information'}})

In [129]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [130]:
# Note: considering from the standpoint of `Extraction procedure`, we do not require
# the `people` item to be specified, what we want is just a list of dictionaries
# constituting the function calls
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]}

In [131]:
# looks for particular key in the output and extract only that
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [132]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [133]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]

##### `Doing it for real`
`We can apply tagging to a larger body of text.`
`For example, let's load this blog post and extract tag information from a sub-set of the text`

In [134]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [135]:
doc = documents[0]

In [136]:
page_content = doc.page_content[:10000]
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [137]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [138]:
overview_tagging_functions = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_functions,
    function_call={"name": "Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [139]:
tagging_chain.invoke({"input": page_content})

{'summary': 'This article discusses the concept of building autonomous agents powered by LLM (large language model) as their core controller. It explores the key components of such agent systems, including planning, memory, and tool use. It also covers various techniques for task decomposition and self-reflection in autonomous agents. The article provides examples of case studies and challenges in implementing LLM-powered agents.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, case studies, challenges'}

In [140]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]

class Info(BaseModel):
    """Information to extract."""
    papers: List[Paper]

In [141]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function,
    function_call={"name": "Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [142]:
extraction_chain.invoke({"input": page_content})
# this is actually title we're passing in, we don't want that, it should rather extract the papers mentioned within.

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [143]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article.

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [144]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [145]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': 'Wei et al.'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al.'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al.'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al.'}]

In [146]:
extraction_chain.invoke({"input": "hi"}) # based on the instruction from prompt `return empty list`

[]

In [147]:
# what is we want to extract all the papers mentioned in the whole article.
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [148]:
splits = text_splitter.split_text(doc.page_content)

In [149]:
len(splits)

14

In [150]:
# Given the above proc., we are gonna try to automatize the above proc. + extraction + output-parsing
# i. We're gonna take the doc.page_content and split it up into chunks
# ii. pass individual split to the extraction chain defined above
# iii. finally, join all the results

In [151]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [152]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [153]:
# i. (a) we need to create some-method of split the page-content and passing it into the chain.
# for that we require each split (i.e. a string chunk) to be fed as dictionary with the "input" key-value
from langchain.schema.runnable import RunnableLambda

In [154]:
# RunnableLambda is simple wrapper in LangChain that takes in a lambda function and converts to Runnable object
prep = RunnableLambda(
    lambda x: [{"input": doc_chunk} for doc_chunk in text_splitter.split_text(x)]
)

In [155]:
prep.invoke("hi")

[{'input': 'hi'}]

In [156]:
# NOTE: extraction_chain operates on single, and here we have list of elements,
# .map() -> take previous input (i.e. list) and map the chain (i.e. extraction_chain) over them.
# Additionally, since `flatten` is not the first component so not compulsory to wrap it into a
# RunnableLambda (but we can its optional, works either way)
chain = prep | extraction_chain.map() | flatten

In [159]:
# chain.invoke(doc.page_content)

#### `Tools and Routing`

- `Create your own tools`
- `Build a tool based on an OpenAPI spec`
    - `Predating LLMs, the OpenAPI specification is routinely used by servie providers to describe the APIs.`
- `Select from multiple possible tools - called "rounting"`

In [160]:
# tool is a decorator from langchain, which can be put on top of a function
# which will automatically convert the function into a langchain tool.
from langchain.agents import tool

In [161]:
@tool
def search(query: str) -> str:
    """"Search for weather online"""
    return "42f"

In [162]:
search.name

'search'

In [163]:
search.description

'search(query: str) -> str - "Search for weather online'

In [164]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [165]:
# we can improve upon this by defining an explicit structure for input schema
# because, the description of the input is what LLM uses to determine what the input should be.
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [166]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [167]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [168]:
# still callable!
search.run("sf")

'42f'

In [169]:
# i. create a tool which gets the temperature, given the latitude and longitude.

In [170]:
import requests
from pydantic import BaseModel, Field
import datetime

# define the input schema
# NOTE: `...` is used to specific that the field is required
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # parameters for the request
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1
    }

    # make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results["hourly"]["temperature_2m"]

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f"The current temperature is {current_temperature}°C"

In [171]:
get_current_temperature.name

'get_current_temperature'

In [172]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [173]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [174]:
# we can convert the tool into the exact OpenAI required JSON blob for function definition
from langchain.tools.render import format_tool_to_openai_function

In [175]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'title': 'OpenMeteoInput',
  'type': 'object',
  'properties': {'latitude': {'title': 'Latitude',
    'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'title': 'Longitude',
    'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [176]:
# this tool is callable
get_current_temperature({
    "latitude": 13,
    "longitude": 14
})

'The current temperature is 21.5°C'

In [177]:
# ii. create a wikipedia tool and it can search things on wikipedia

In [178]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries"""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [179]:
search_wikipedia.name

'search_wikipedia'

In [180]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries'

In [181]:
search_wikipedia.args

{'query': {'title': 'Query', 'type': 'string'}}

In [182]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries',
 'parameters': {'title': 'search_wikipediaSchemaSchema',
  'type': 'object',
  'properties': {'query': {'title': 'Query', 'type': 'string'}},
  'required': ['query']}}

In [183]:
search_wikipedia({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: OpenAI\nSummary: OpenAI is an American artificial intelligence (AI) research organization consisting of the non-profit OpenAI, Inc. registered in Delaware and its for-profit subsidiary OpenAI Global, LLC. OpenAI researches artificial intelligence with the declared intention of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work". OpenAI has also developed several large language models, such as ChatGPT and GPT-4, as well as advanced image generation models like DALL-E 3, and in the past published open-source models.The organizati

In [184]:
# previously, we have created function (defined them) and then created OpenAI function definitions
# for those functions.
# Often time, function which we want to interact with are exposed through APIs and often times, APIs
# a specific specification for the inputs and outputs called as OpenAPI Specification.

# Below, we take a example OpenAPI Sepcification and convert it into a OpenAI function definitions

In [185]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn # openapispec -> openai_fn
from langchain.utilities.openapi import OpenAPISpec # used to load OpenAPISpec in the first place

In [186]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [187]:
# ## Below code is not working, Google Colab had some issue running it, might be OpenAPI class problem "`super` object has no attribute `parse_obj`""
# # load OpenAPISpec from the above text
# spec = OpenAPISpec.from_text(text)

# pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

# from langchain.chat_models import ChatOpenAI
# model = ChatOpenAI(temperature=0.).bind(functions=pet_openai_functions)

# # example runs
# model.invoke("what are three pets names"), model.invoke("tell me about pet with id 42")

##### `Routing`

In [188]:
# Here we're going to use the 2 tools defined above `get_current_temperature` and `search_wikipedia`
# we're gonna make OpenAI decide which tool to select based on their corresponding function definition
# and then we're also going to invoke the tool to get response. => This is called Routing

# Routing: where we used LLM to determine which path to take and also the input to the paths

In [189]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
functions

[{'name': 'search_wikipedia',
  'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries',
  'parameters': {'title': 'search_wikipediaSchemaSchema',
   'type': 'object',
   'properties': {'query': {'title': 'Query', 'type': 'string'}},
   'required': ['query']}},
 {'name': 'get_current_temperature',
  'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
  'parameters': {'title': 'OpenMeteoInput',
   'type': 'object',
   'properties': {'latitude': {'title': 'Latitude',
     'description': 'Latitude of the location to fetch weather data for',
     'type': 'number'},
    'longitude': {'title': 'Longitude',
     'description': 'Longitude of the location to fetch weather data for',
     'type': 'number'}},
   'required': ['latitude', 'longitude']}}]

In [190]:
from langchain.chat_models import ChatOpenAI

In [191]:
model = ChatOpenAI(temperature=0.).bind(functions=functions)

In [192]:
model.invoke("what is the weather in sf right now?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})

In [193]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "langchain"\n}', 'name': 'search_wikipedia'}})

In [194]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}")
])
chain = prompt | model

In [195]:
chain.invoke({"input": "what is the weather in sf right now?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})

In [196]:
# this will is going to take in output and parse that to determine whether its a function call
# or a response and if it a function call, what function is to be called and what's input
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [197]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [198]:
result = chain.invoke({"input": "what is the weather in sf right now?"})

In [199]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [200]:
result.tool

'get_current_temperature'

In [201]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [202]:
get_current_temperature(result.tool_input)

'The current temperature is 17.5°C'

In [203]:
result = chain.invoke({"input": "hi!"})

In [204]:
type(result)

langchain.schema.agent.AgentFinish

In [205]:
result.return_values # `.return_values` is common to all AgentFinish response types

{'output': 'Hello! How can I assist you today?'}

In [206]:
# define Route function, it will act on the model output and decide either scenarios
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values["output"]
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature
        }
        return tools[result.tool].run(result.tool_input) # apply the response to the callable

In [207]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [208]:
result = chain.invoke({"input": "what is the weather in san fransisco now?"})

In [209]:
result

'The current temperature is 17.5°C'

In [210]:
result = chain.invoke({"input": "what is langchain?"})

In [211]:
result

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: OpenAI\nSummary: OpenAI is an American artificial intelligence (AI) research organization consisting of the non-profit OpenAI, Inc. registered in Delaware and its for-profit subsidiary OpenAI Global, LLC. OpenAI researches artificial intelligence with the declared intention of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work". OpenAI has also developed several large language models, such as ChatGPT and GPT-4, as well as advanced image generation models like DALL-E 3, and in the past published open-source models.The organizati

In [212]:
chain.invoke({"input": "hi!"})

'Hello! How can I assist you today?'

#### `Conversational Agent`

`This will comprise tool usage and chat memory to mimic something like ChatGPT`

- `Build some tools`
- `Write your own agent loop using LCEL`
- `Utilize agent_executor which:`
    - `Implements the agent loop`
    - `Adds error handling, early stopping, tracing etc.`

In [213]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [214]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [215]:
tools = [get_current_temperature, search_wikipedia]

In [216]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [217]:
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [218]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [219]:
result.tool

'get_current_temperature'

In [220]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [221]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # place where we can pass a list of observation retrieved after utilization of certain tool
])

In [222]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [223]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [224]:
result1.tool

'get_current_temperature'

In [225]:
observation = get_current_temperature(result1.tool_input)

In [226]:
observation

'The current temperature is 17.5°C'

In [227]:
type(result1)

langchain.schema.agent.AgentActionMessageLog

In [228]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [229]:
result1.message_log # contains list of messages that makes up how we arrive at current AgentAction

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})]

In [230]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}}),
 FunctionMessage(content='The current temperature is 17.5°C', name='get_current_temperature')]

In [231]:
result2 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [232]:
result2 # `AgentFinish`

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 17.5°C.'}, log='The current temperature in San Francisco is 17.5°C.')

In [233]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [234]:
# `RunnablePassthrough` takes an inital input and passes it through
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [235]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [236]:
run_agent("what is the weather is sf?")

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 17.5°C.'}, log='The current temperature in San Francisco is 17.5°C.')

In [237]:
run_agent("what is langchain?")

AgentFinish(return_values={'output': 'I couldn\'t find specific information about "LangChain" in my search results. It\'s possible that LangChain is a term or concept that is not widely known or documented. If you have any additional information or context about LangChain, I may be able to provide more assistance.'}, log='I couldn\'t find specific information about "LangChain" in my search results. It\'s possible that LangChain is a term or concept that is not widely known or documented. If you have any additional information or context about LangChain, I may be able to provide more assistance.')

In [238]:
run_agent("hi!")

AgentFinish(return_values={'output': 'Hello! How can I assist you today?'}, log='Hello! How can I assist you today?')

In [239]:
# there is a class in LangChain which work similar to the above defined `run_agent` function which loops over responses
# of the LLMs and make observations and decide upon further. Additionally, it has better logging mechanism, error handling for LLMs and tools

In [240]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [241]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Prompt engineering
Summary: Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model. A prompt is natural language text describing the task that an AI should perform.A prompt for a text-to-text model can be a query such as "what is Fermat's little theorem?", a command such as "write a poem about leaves falling", a short statement of feedback (for example, "too verbose", "too formal", "rephrase again", "omit this word") or a longer statement including context, instructions, and input data. P

{'input': 'what is LangChain?',
 'output': "LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework that can be used for various tasks such as document analysis and summarization, chatbots, and code analysis. LangChain's use-cases largely overlap with those of language models in general."}

In [242]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob', 'output': 'Hello Bob! How can I assist you today?'}

In [243]:
agent_executor.invoke({"input": "what is my name"}) # this is since there is not memory factor for previous message



> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to personal information.

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, but I don't have access to personal information."}

In [244]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [245]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [246]:
from langchain.memory import ConversationBufferMemory
# `return_messages=True` argument makes sure that the returned messages are appended in form of a list
# since if `False` it would return as string which is not ideal for MessagesPlaceholder
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [247]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [248]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Hello Bob! How can I assist you today?'}

In [249]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.')],
 'output': 'Your name is Bob.'}

In [250]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 17.5°CThe current temperature in San Francisco is 17.5°C.

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content='The current temperature in San Francisco is 17.5°C.')],
 'output': 'The current temperature in San Francisco is 17.5°C.'}

`Additional module to Create a chatbot, which utilizes panel library to create a chat interface and utilizes the AgentExecutor class to mimic a chatagent and then converse with user.`